# F4. AI-Driven Mitigation Recommendations

## Intended Purpose of Code

This feature enhances the risk scoring dashboard by integrating openAI generated recommendations based on the calculations and pre-analysis generated in the risk scoring dashboard.

### Key Features

1. Explain My Top Risk/Mitigations Tips
    * prompt openAI to summarize in a numbered list
        * the users top risks
        * mitigation recommendations to harden systems
3. Risk Trend Insights
    * Calculate quarter over quarter change trends
        * aggregates counts of CVEs by asset and severity
            * grouped by quarter or year
        * groups trends by by period, asset, and severity
    * Prompt openAI to
        * analyze the trend data
        * highlight emerging risk patterns
        * narrate where there were significant spikes or drops
        * call out assets with the most notable changes

In [2]:
from openai import OpenAI
import os
import pandas as pd
from IPython.display import Markdown, display
import json

# Config
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("API key not set.")
client = OpenAI(api_key=api_key)
asset_scores = pd.read_csv('../data/asset_risk_summary.csv')
vuln_scores = pd.read_csv('../data/cve_vuln_summary.csv')
vul_catalogue = pd.read_csv('../data/vuln_catalogue_v2.csv')

# Tidy up
scores = pd.merge(asset_scores,vuln_scores,how='inner',on=['cpeName','Title'])
scores.sort_values(by='riskScore').head(10)
df0 = pd.merge(vul_catalogue,scores, how='inner', on=['Title','cpeName','cveID'])
df0.drop(columns=['Unnamed: 0','vectorString','WrittenAt'],inplace=True,axis=1)
df = df0.sort_values(by='riskScore', ascending=False)
top10 = df.head(5)

In [5]:
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Title                     442 non-null    object 
 1   cpeName                   442 non-null    object 
 2   MaxRiskScore              440 non-null    float64
 3   countHighRiskCVEs (>7.0)  442 non-null    int64  
 4   cveID                     440 non-null    object 
 5   riskScore                 421 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 20.8+ KB


In [4]:
# Explain My Top Risks & Mitigation Tips Prep
cols = (['Title',
        'cpeName',
        'cveID',
        'published',
        'baseScore',
        'exploitabilityScore',
        'impactScore',
        'riskScore',
        'MaxRiskScore',
        'countHighRiskCVEs (>7.0)',
        'baseSeverity',
        'attackVector',
        'attackComplexity',
        'confidentialityImpact',
        'availabilityImpact',
        'description'])
records = top10[cols].to_dict(orient='records')

def explain_top_risks(df, client):
    risks_json = json.dumps(records, indent=2)

    # Construct the system and user prompts
    system_prompt = (
        "You are a cybersecurity expert specializing in vulnerability risk analysis "
        "and mitigation planning with a focus on NIST's Cybersecurity Framework 2.0."
    )

    user_prompt = (
        "Below is a list of the 10 highest-risk vulnerabilities (CVEs) affecting my assets. "
        "For each item, do the following:\n"
        "1. Briefly explain why this CVE poses a high risk based on its details.\n"
        "2. Suggest concise, actionable mitigation steps or best practices.\n"
        "Generate a report memo with only the subject line in the memo header,"
        "Present the output as a numbered list. Here are the vulnerabilities:\n"
        f"{risks_json}"
    )

    # Send to OpenAI
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1000,
        temperature=0.2
    )
    return response.choices[0].message.content

# Now, call the function and store the result
top_risks = explain_top_risks(df, client)

with open("../markdown/top_risks.md", "w", encoding="utf-8") as f:
    f.write(top_risks)

with open("../markdown/top_risks.md", "r", encoding="utf-8") as f:
    md_content = f.read()

display(Markdown(md_content))

**Subject: High-Risk Vulnerability Analysis and Mitigation Recommendations**

1. **CVE-2020-1953 - Oracle Database Server 19c**
   - **Risk Explanation:** This vulnerability allows remote code execution due to improper handling of YAML files, which can lead to unauthorized code execution if YAML files are loaded from untrusted sources.
   - **Mitigation Steps:** 
     - Update Apache Commons Configuration to a version that addresses this vulnerability.
     - Ensure YAML files are only loaded from trusted sources.
     - Implement input validation and sanitization for all external data sources.

2. **CVE-2024-21410 - Microsoft Exchange Server 2019 Cumulative Update 14**
   - **Risk Explanation:** This vulnerability allows for elevation of privilege, potentially giving attackers unauthorized access to sensitive data and system controls.
   - **Mitigation Steps:**
     - Apply the latest security patches from Microsoft.
     - Regularly review and update user permissions to ensure least privilege.
     - Monitor Exchange Server logs for unusual activity.

3. **CVE-2019-0586 - Microsoft Exchange Server 2019**
   - **Risk Explanation:** This vulnerability involves remote code execution due to improper memory handling, which can be exploited to execute arbitrary code on the server.
   - **Mitigation Steps:**
     - Install the latest security updates from Microsoft.
     - Implement network segmentation to limit exposure of Exchange servers.
     - Conduct regular security audits and penetration testing.

4. **CVE-2023-27997 - Fortinet FortiGate 7000**
   - **Risk Explanation:** A heap-based buffer overflow vulnerability that allows remote attackers to execute arbitrary code via crafted requests, compromising system integrity.
   - **Mitigation Steps:**
     - Upgrade to the latest version of FortiOS and FortiProxy that addresses this vulnerability.
     - Enable and configure intrusion prevention systems (IPS) to detect and block exploit attempts.
     - Regularly review and update firewall rules and configurations.

5. **CVE-2019-16942 - Oracle Database Server 19c**
   - **Risk Explanation:** This vulnerability can lead to remote code execution through polymorphic typing issues in JSON endpoints, potentially allowing attackers to execute malicious payloads.
   - **Mitigation Steps:**
     - Update FasterXML jackson-databind to a secure version.
     - Disable Default Typing in JSON endpoints unless absolutely necessary.
     - Conduct regular code reviews and security assessments to identify and mitigate similar vulnerabilities.

In [6]:
# Updated to support user-defined date range input for trend analysis

from openai import OpenAI
import os
import pandas as pd
from IPython.display import Markdown, display
import json

# Config
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("API key not set.")
client = OpenAI(api_key=api_key)

# Load data
asset_scores = pd.read_csv('../data/asset_risk_summary.csv')
vuln_scores = pd.read_csv('../data/cve_vuln_summary.csv')
vul_catalogue = pd.read_csv('../data/vuln_catalogue_v2.csv')

# Merge data
scores = pd.merge(asset_scores, vuln_scores, how='inner', on=['cpeName', 'Title'])
df0 = pd.merge(vul_catalogue, scores, how='inner', on=['Title', 'cpeName', 'cveID'])
df0.drop(columns=['Unnamed: 0', 'vectorString', 'WrittenAt'], inplace=True)
df = df0.sort_values(by='riskScore', ascending=False)

# Define functions
def get_trend_summary(df, freq='Q'):
    df['published'] = pd.to_datetime(df['published'])
    if freq == 'Q':
        df['period'] = df['published'].dt.to_period('Q')
    elif freq == 'Y':
        df['period'] = df['published'].dt.to_period('Y')
    else:
        raise ValueError("freq must be 'Q' for quarter or 'Y' for year.")
    trend = df.groupby(['period', 'cpeName', 'baseSeverity']).size().unstack(fill_value=0)
    return trend

def calculate_qoq_change(trend_df):
    trend_df = trend_df.sort_index(level=0)
    pct_change = trend_df.groupby('cpeName').pct_change().replace([float('inf'), -float('inf')], 0).fillna(0) * 100
    return pct_change.round(1)

def generate_trend_narrative_prompt(trend_df, pct_change_df, start_period, end_period):
    filtered_trend = trend_df.loc[start_period:end_period].reset_index()
    filtered_pct_change = pct_change_df.loc[start_period:end_period].reset_index()

    for df in (filtered_trend, filtered_pct_change):
        if 'period' in df.columns:
            df['period'] = df['period'].astype(str)

    trend_data = filtered_trend.to_dict(orient='records')
    change_data = filtered_pct_change.to_dict(orient='records')

    prompt = (
        "You are a cybersecurity risk analyst. Below are two tables:\n\n"
        "1. Raw counts of vulnerabilities (CVEs) by asset and severity for each quarter.\n"
        "2. The corresponding quarter-over-quarter percentage change for each asset and severity.\n\n"
        "Analyze the data, highlighting emerging risk patterns. Narrate where there were significant spikes or drops,\n"
        "and call out assets with the most notable changes. Use percentages and time periods\n"
        "(e.g., 'Web servers saw a 40% spike in Critical CVEs in Q1 2025').\n"
        "Your response should be titled: Risk Trends Insights.\n"
        "Finish by suggesting which assets or severities require immediate attention due to recent trends.\n\n"
        f"Raw trend data:\n{json.dumps(trend_data, indent=2)}\n\n"
        f"Quarterly percent change data:\n{json.dumps(change_data, indent=2)}\n"
    )
    return prompt

# Generate trends and changes
quarterly_trend = get_trend_summary(df, freq='Q')
quarterly_change = calculate_qoq_change(quarterly_trend)

# --- User input for date range ---
start_period = input("Enter start quarter (e.g., 2023Q1): ")
end_period = input("Enter end quarter (e.g., 2024Q4): ")

# --- Generate prompt and call OpenAI ---
prompt = generate_trend_narrative_prompt(quarterly_trend, quarterly_change, start_period, end_period)
system_prompt = "You are an expert at analyzing vulnerability data for security reporting with a particular interest in NIST's Cybersecurity Framework 2.0."

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    max_tokens=1000,
    temperature=0.3
)

risk_trend_narrative = response.choices[0].message.content

# Save and display
with open("../markdown/risk_trend_narrative.md", "w", encoding="utf-8") as f:
    f.write(risk_trend_narrative)

with open("../markdown/risk_trend_narrative.md", "r", encoding="utf-8") as f:
    md_content = f.read()

display(Markdown(md_content))

Enter start quarter (e.g., 2023Q1):  2022Q1
Enter end quarter (e.g., 2024Q4):  2025Q2


# Risk Trends Insights

The analysis of the vulnerability data from 2022Q1 to 2025Q1 reveals several noteworthy trends and risk patterns across different assets and severity levels.

### Key Observations:

1. **Adobe Acrobat Reader:**
   - **High Severity CVEs:** Adobe Acrobat Reader consistently showed fluctuations in high-severity vulnerabilities. Notably, there was a significant spike of 466.7% in 2022Q1, followed by another increase of 111.8% in 2022Q2. However, this was followed by a sharp decline of 52.8% in 2022Q3 and 88.2% in 2022Q4. In 2023Q1, there was a resurgence with a 550% increase, indicating persistent volatility.
   - **Medium Severity CVEs:** Medium-severity vulnerabilities also exhibited volatility, with a 300% increase in 2022Q2 and a dramatic 1800% increase in 2023Q3. However, there were significant drops in other quarters, such as a 75% decrease in 2023Q2 and 57.9% in 2023Q4.
   - **Overall Trend:** Adobe Acrobat Reader's vulnerabilities, particularly in high and medium severities, show a pattern of instability, suggesting a need for continuous monitoring and patch management.

2. **Oracle Database and Database Server:**
   - **High Severity CVEs:** The Oracle Database showed a significant drop in high-severity vulnerabilities, with a 100% decrease in 2022Q3 and no high-severity vulnerabilities reported in subsequent quarters.
   - **Medium Severity CVEs:** There was a notable increase of 350% in medium-severity vulnerabilities in 2022Q3, but this was followed by a complete drop to zero in 2022Q4.
   - **Overall Trend:** The Oracle Database and Database Server showed a reduction in vulnerabilities over time, but the initial spikes in medium severity in 2022Q3 highlight areas that required immediate attention at that time.

3. **Microsoft Exchange Server:**
   - **Critical Severity CVEs:** In 2023Q3, there was a critical vulnerability reported, which was resolved by 2024Q1.
   - **Overall Trend:** The presence of critical vulnerabilities in Microsoft Exchange Server, although resolved, indicates the need for proactive measures to prevent future occurrences.

4. **Fortinet FortiGate 7000:**
   - **Critical Severity CVEs:** A critical vulnerability was reported in 2023Q2, emphasizing the importance of addressing such high-risk issues promptly.

### Emerging Risk Patterns:

- **Volatility in Adobe Acrobat Reader:** The frequent and significant changes in high and medium-severity vulnerabilities suggest that Adobe Acrobat Reader is a high-risk asset that requires continuous attention and timely patching.
- **Stability in Oracle Products:** While Oracle products showed initial spikes, the subsequent stabilization indicates effective vulnerability management, but vigilance is still necessary to maintain this trend.
- **Critical Vulnerabilities in Microsoft and Fortinet Products:** The presence of critical vulnerabilities in Microsoft Exchange Server and Fortinet FortiGate 7000 highlights the need for immediate remediation and ongoing security assessments.

### Recommendations for Immediate Attention:

- **Adobe Acrobat Reader:** Given the persistent volatility in vulnerabilities, especially in high and medium severities, it is crucial to prioritize patch management and security updates for Adobe Acrobat Reader.
- **Critical Vulnerabilities in Microsoft Exchange and Fortinet FortiGate:** These assets should be closely monitored for any new vulnerabilities, and existing ones should be addressed with urgency to mitigate potential exploitation risks.

In conclusion, while some assets have shown improvement in vulnerability management, others, particularly Adobe Acrobat Reader, continue to pose significant risks that require ongoing attention and proactive security measures.